# Source Localization with MNE/dSPM/sLORETA/eLORETA

### Import the libraries

In [ ]:
from Preprocess import *
from Plot import *
from Modelling import *
from Statistics import *

from mayavi import mlab
mlab.init_notebook()

%matplotlib inline

### Obtain the directories of subject and FS_average

In [ ]:
trans, src, bem, subjects_dir = get_FS_data()

print('\ntran directory: ', trans)
print('src directory: ', src)
print('bem directory: ', bem)
print('subjects_dir directory: ', subjects_dir)

### Load all the raw data for both cursor and robot

In [ ]:
subject_to_process = ['s02', 's03', 's04', 's05', 's06', 's07', 's08', 's09', 's10', 's11', 's12', 's13']

# Load the raw data for cursor
dataset = 'cursor'
raw_data_list_cursor, events_list_cursor = load_raw_data(subject_to_process, dataset)

# Load the raw data for robot
dataset = 'robot'
raw_data_list_robot, events_list_robot = load_raw_data(subject_to_process, dataset)

### Obtain the raw data and the events

In [ ]:
# Obtain the raw event dictionary for cursor
raw_event_dict_cursor = obtain_data(subject_to_process, raw_data_list_cursor, events_list_cursor)

# Obtain the raw event dictionary for robot
raw_event_dict_robot = obtain_data(subject_to_process, raw_data_list_robot, events_list_robot)

### Create epochs for subjects

In [ ]:
# Obtain the data for cursor
data_cursor = create_epochs(raw_event_dict_cursor, subject_to_process, tmin=-0.2, tmax=0.8, nonError='S  4', error='S  6')

# Obtain the data for robot
data_robot = create_epochs(raw_event_dict_robot, subject_to_process, tmin=-0.2, tmax=0.8, nonError='S  4', error='S  6')

### Create grand average

In [ ]:
# Obtain the grand average cache for cursor
grand_average_cache_cursor = compute_grand_average(data_cursor, subject_to_process)

# Obtain the grand average cache for robot
grand_average_cache_robot = compute_grand_average(data_robot, subject_to_process)

### Plot Cz data

In [ ]:
save = False

# Obtain the Cz channel data
cz_data_cache = obtain_cz_cache(grand_average_cache_cursor, grand_average_cache_robot)

# Plot the Cz channel data
plot_cz_data(cz_data_cache, save)

### Generate grand average topomap

In [ ]:
date_type ='contrast'
save = False

# Compute the grand_average_cache again
grand_average_cache_cursor = compute_grand_average(data_cursor, subject_to_process)
grand_average_cache_robot = compute_grand_average(data_robot, subject_to_process)

# Plot the grand average topomap for both cursor and robot
grand_avg_topomap(grand_average_cache_cursor, grand_average_cache_robot, cz_data_cache, date_type, save)

### Plot grand average (Time Montage)

In [ ]:
# Set the grand average parameters
grand_average_parameters = {'data_type': 'contrast',
                            'method': 'eLORETA',
                            'vector': False,
                            'cz_data_cache': cz_data_cache,                            
                            'selected_view': 'medial',
                            'data_type': 'contrast',
                            'hemisphere': 'lh',
                            'save_time_montage': True}

# Plot the time montage for both cursor and robot
plot_time_montage(grand_average_cache_cursor, grand_average_cache_robot, grand_average_parameters, trans, src, bem, subjects_dir)

### Plot peak activation in volume source space

In [ ]:
modelling_parameters = {'pos': 5.0, 'threshold': 1.6,
                        'method': 'eLORETA', 'save': False}

plot_vol_source_space(grand_average_cache_cursor, grand_average_cache_robot, cz_data_cache, modelling_parameters, trans, bem, subjects_dir)

### Save single image

In [ ]:
grand_average_parameters = {'data_type': 'contrast',
                            'method': 'eLORETA',
                            'vector': False,
                            'cz_data_cache': cz_data_cache,
                            'selected_view': 'lateral',
                            'hemisphere': 'rh',
                            'save': True}

save_single_image(grand_average_cache_cursor, grand_average_cache_robot, grand_average_parameters, trans, src, bem, subjects_dir)

### Save view montage

In [ ]:
grand_average_parameters = {'data_type': 'contrast',
                            'method': 'dSPM',
                            'vector': True,
                            'cz_data_cache': cz_data_cache,
                            'hemisphere': 'lh',
                            'save_view_montage': False}

save_view_montage(grand_average_cache_cursor, grand_average_cache_robot, grand_average_parameters, trans, src, bem, subjects_dir)

### Delete data to free up memory

In [ ]:
del raw_event_dict_cursor, raw_event_dict_robot, events_list_cursor, events_list_robot

### Perform Statistical Analysis

In [ ]:
vector = False
method = 'eLORETA'# 'dSPM' 'MNE' 'sLORETA' 'eLORETA'

# Obtain data to perform statistics
data_array, parameter_cache = obtain_statistical_data(data_cursor, data_robot, subject_to_process, method, vector, trans, src, bem)

In [ ]:
# Morph the subject brain to fs average brain
morphed_data, parameter_cache = morph_data(data_array, parameter_cache, vector, subjects_dir)
del data_array

In [ ]:
# Perform statistics on the morphed data
clu, good_cluster_inds = perform_statistics(morphed_data, parameter_cache, vector, p_value = 0.001)

### Visualize the Statistics

In [ ]:
epsilon = {'1_min': 15, '1_max': 15, '2_min': 10, '2_max': 10, '3_min': 20, '3_max': 30, '4_min': 40, '4_max': 40}
           
modelling_parameters = {'hemisphere': 'lh', 'selected_view': 'medial',
                        'save': True, 'epsilon': epsilon_list,
                        'vector': False}

visualize_cluster(clu, cz_data_cache, parameter_cache, modelling_parameters, subjects_dir)